In [ ]:
!if ! test -f beeradvocate.json.gz; then wget https://datarepo.eng.ucsd.edu/mcauley_group/data/beer/beeradvocate.json.gz; fi;
# !gzip -d beeradvocate.json.gz

!pip install tqdm

In [ ]:
import gzip
from tqdm import tqdm
import numpy as np
import string
from collections import defaultdict
from sklearn import linear_model
import gensim
from gensim.models import Doc2Vec

In [ ]:
fpath = "beeradvocate.json.gz"

def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf-8"):
        yield eval(l)

data = []
for l in readGz(fpath):
    data.append(l)
    if len(data) >= 100000:
        break

In [ ]:
n = len(data)

dataTrain = data[:int(n*0.8)]
dataVal = data[int(n*0.8):int(n*0.9)]
dataTest = data[int(n*0.9):]

In [ ]:
dataTrain[0]

In [ ]:
def get_corpus(data):
  for i, d in enumerate(data):
    tokens = gensim.utils.simple_preprocess(d['review/text'])
    yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [ ]:
train_corpus = list(get_corpus(dataTrain))
val_corpus = list(get_corpus(dataVal))

In [ ]:
print(train_corpus[0])

In [ ]:
model = Doc2Vec(vector_size=100, min_count=2, epochs=50)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
Xtrain = [model.infer_vector(gensim.utils.simple_preprocess(d['review/text'])) for d in dataTrain]
Xval = [model.infer_vector(gensim.utils.simple_preprocess(d['review/text'])) for d in dataVal]

In [ ]:
ytrain = [d['review/overall'] for d in dataTrain]
yval   = [d['review/overall'] for d in dataVal]

In [ ]:
mod = linear_model.LogisticRegression(C=1)
mod.fit(Xtrain, ytrain)
ypreds = mod.predict(Xval)
correct = [int(yp == y) for yp, y in zip(ypreds, yval)]
acc = sum(correct) / len(correct)

In [ ]:
mod.score(Xval, yval)

In [ ]:
ypreds[:10]

In [ ]:
Xtrain[0]